In [1]:
import os
import numpy as np
import pandas as pd
from random import random

from agroplot import GoogleMapPlotter

import webbrowser


In [2]:
# Rutas
ROOT_PATH = 'C:/Users/pedro/ds-geo'
CONFIG_PATH = ROOT_PATH + '/config'
MAPS_PATH = ROOT_PATH + '/maps'
DATA_PATH = ROOT_PATH + '/data'

os.chdir(ROOT_PATH)

# Valores por defecto 
INITIAL_LAT = 39.690578   # 39.720578 
INITIAL_LONG = -2.924675 
INITIAL_ZOOM = 13


# GoogleMaps

In [3]:
# Adaptación de la clase GoogleMapPlotter
# Se añaden las funciones AddMap para visualizar sobre el mapa los elementos de los datasets

class CustomGoogleMapPlotter(GoogleMapPlotter):
    
    # __init__
    def __init__(self, center_lat, center_lng, zoom, apikey='',
                 map_type='satellite'):
        if apikey == '':
            try:
                with open(CONFIG_PATH + '/apikey.txt', 'r') as apifile:
                    apikey = apifile.readline()
            except FileNotFoundError:
                pass
        super().__init__(center_lat, center_lng, zoom, apikey=apikey)

        self.map_type = map_type
        assert(self.map_type in ['roadmap', 'satellite', 'hybrid', 'terrain'])
        

    # write_map
    def write_map(self,  f):
        f.write('\t\tvar centerlatlng = new google.maps.LatLng(%f, %f);\n' %
                (self.center[0], self.center[1]))
        f.write('\t\tvar myOptions = {\n')
        f.write('\t\t\tzoom: %d,\n' % (self.zoom))
        f.write('\t\t\tcenter: centerlatlng,\n')

        # Change this line to allow different map types
        f.write('\t\t\tmapTypeId: \'{}\'\n'.format(self.map_type))
        f.write('\t\t};\n')
        f.write('\t\tvar map = new google.maps.Map(document.getElementById("map_canvas"), myOptions);\n')
        f.write('\n')


    # Añade polígonos y parcelas al mapa
    def AddMapPolygons(self, df, mode=None):

        for index, row in df.iterrows():
            l_lat = row['Latitude']
            l_long = row['Longitude']

            if mode == 'c':
                s_color = row['ColorCultivo']
                if s_color == '':
                    s_color = 'white'
            elif mode == 'p':
                s_color = row['ColorPropietario']
                if s_color == '':
                    s_color = 'white'
            else:
                s_color = 'cornflowerblue'
            
            dict_info = {}
            dict_info.update({'descripcion': row['NombreParcela']})
            dict_info.update({'tipo': row['NombreCultivo']})
            dict_info.update({'tipo_color': row['ColorCultivo']})
            dict_info.update({'variedad': row['VariedadCultivo']})
            dict_info.update({'municipio': row['CatMunicipio']})
            dict_info.update({'poligono': str(row['CatPoligono'])})
            dict_info.update({'parcela': str(row['CatParcela'])})
            dict_info.update({'recinto': str(row['CatRecinto'])})
            dict_info.update({'propietario': row['NombrePropietario']})
            dict_info.update({'propietario_color': row['ColorPropietario']})

            self.polygon(l_lat, l_long, info=dict_info, color=s_color, face_alpha=0.5) 

        
    # Añade puntos al mapa
    def AddMapPoints(self, df, mode=None):

        for index, row in df.iterrows():
            f_lat = row['Latitude']
            f_long = row['Longitude']
            s_title = row['Etiqueta'] 
                           
            if mode == 'c':
                s_color = row['ColorCultivo']
                if s_color == '':
                    s_color = 'white'
            elif mode == 'p':
                s_color = row['ColorPropietario']
                if s_color == '':
                    s_color = 'white'
            else:
                s_color = 'cornflowerblue'
            
            dict_info = {}
            dict_info.update({'descripcion': s_title})
            dict_info.update({'tipo': row['NombreCultivo']})
            dict_info.update({'tipo_color': row['ColorCultivo']})
            dict_info.update({'propietario': row['NombrePropietario']})
            dict_info.update({'propietario_color': row['ColorPropietario']})

            self.circle(f_lat, f_long, radius=10, info=dict_info, color=s_color, alpha=0.6)

            
    # Añade al mapa las líneas, trazados, rutas, ..        
    def AddMapLines(self, df, mode=None):
        
        l_origenes = df['Origen'].unique()
        for origen in l_origenes:
            df_parcial = df[df['Origen'] == origen]

            f_lat = df_parcial['Latitude']
            f_long = df_parcial['Longitude']
            s_color = df_parcial['ColorCultivo'].iloc[0]

            if mode == 'c':
                if s_color == '': s_color = 'white'
            elif mode == 'p':
                s_color = 'white'
            else:
                s_color = 'cornflowerblue'

            self.plot(f_lat, f_long, color=s_color, edge_width=2)

        
    # Añade marcadores
    def AddMapMarkers(self, df, mode=None):

        for index, row in df.iterrows():
            f_lat = row['Latitude']
            f_long = row['Longitude']
            s_title = row['Etiqueta'] 
                            
            if mode == 'c':
                s_color = row['ColorCultivo']
                if s_color == '':
                    s_color = 'white'
            elif mode == 'p':
                s_color = row['ColorPropietario']
                if s_color == '':
                    s_color = 'white'
            else:
                s_color = 'cornflowerblue'
                
            dict_info = {}
            dict_info.update({'descripcion': s_title})
            dict_info.update({'tipo': row['NombreCultivo']})
            dict_info.update({'tipo_color': row['ColorCultivo']})
            dict_info.update({'propietario': row['NombrePropietario']})
            dict_info.update({'propietario_color': row['ColorPropietario']})
            dict_info.update({'latitud': str(round(f_lat, 6))})
            dict_info.update({'longitud': str(round(f_long, 6))})
            
            self.marker(f_lat, f_long, title=s_title, info=dict_info, color=s_color) 

    

In [4]:
# Funciones para cargar los datasets previamente preparados

# Función que carga el dataset de las parcelas
def LoadDatasetsParcels(layers):

    # Defino el formato del dataframe
    df_parcels = pd.DataFrame(columns=['CatReferencia', 'CatRecinto', 'Latitude', 'Longitude', 'CatMunicipio', 'CatPoligono', 'CatParcela', 'VariedadCultivo', 'NombreParcela', 'NombreCultivo', 'ColorCultivo', 'NombrePropietario', 'ColorPropietario', 'Origen'])
    
    # Cargo todos los datasets de los layers
    for layer in layers:
        s_file = 'dataset-parcels.pkl'
        s_path = DATA_PATH + '/layer-' + layer
        if (os.path.isdir(s_path)) & (os.path.isfile(s_path + '/parcels.xls')):
            df_file = pd.read_pickle(s_path + '/' + s_file)
            df_parcels = pd.concat([df_parcels, df_file], axis=0, ignore_index=True)   
    
    # Limpio el dataframe final
    df_parcels.fillna('', inplace=True)
    
    return df_parcels


# Función que carga el dataset de los puntos de interés
def LoadDatasetsPoints(layers):
    
    # Defino el formato del dataframe
    df_points = pd.DataFrame(columns=['Latitude', 'Longitude', 'Tipo', 'Etiqueta', 'NombreCultivo', 'ColorCultivo', 'Origen'])
    
    # Cargo todos los datasets de los layers
    for layer in layers:
        s_file = 'dataset-points.pkl'
        s_path = DATA_PATH + '/layer-' + layer
        if (os.path.isdir(s_path)) & (os.path.isfile(s_path + '/points.xls')):
            df_file = pd.read_pickle(s_path + '/' + s_file)
            df_points = pd.concat([df_points, df_file], axis=0, ignore_index=True)   
    
    # Limpio el dataframe final
    df_points.fillna('', inplace=True)
    df_points.drop_duplicates(inplace=True, ignore_index=True)
    
    return df_points


# Función que carga el dataset de los markers
def LoadDatasetsMarkers(layers):
    
    # Defino el formato del dataframe
    df_markers = pd.DataFrame(columns=['Latitude', 'Longitude', 'Tipo', 'Etiqueta', 'NombreCultivo', 'ColorCultivo', 'Origen'])
    
    # Cargo todos los datasets de los layers
    for layer in layers:
        s_file = 'dataset-markers.pkl'
        s_path = DATA_PATH + '/layer-' + layer
        if (os.path.isdir(s_path)) & (os.path.isfile(s_path + '/markers.xls')):
            df_file = pd.read_pickle(s_path + '/' + s_file)
            df_markers = pd.concat([df_markers, df_file], axis=0, ignore_index=True)   
    
    # Limpio el dataframe final
    df_markers.fillna('', inplace=True)
    df_markers.drop_duplicates(inplace=True, ignore_index=True)
    
    return df_markers


# Función que carga el dataset de los markers
def LoadDatasetMyTrack():
    df_points = pd.read_pickle(DATA_PATH + '/' + DATASET_MYTRACK_P)
    df_markers = pd.read_pickle(DATA_PATH + '/' + DATASET_MYTRACK_M)
    return (df_points, df_markers)


def LoadDatasets(layers):

    df_parcelas = LoadDatasetParcelas()
    df_puntos = LoadDatasetPuntos()
    df_markers = LoadDatasetsMarkers(layers)
    
    return (df_parcelas, df_puntos, df_markers)


In [5]:
# Cálculo del centro del mapa y del zoom inicial (para los ficheros de parcelas KML)

def GetMapInitialSizeKML(df):
    
    max_lat = df['Latitude'].apply(max).max()
    min_lat = df['Latitude'].apply(min).min()
    max_long = df['Longitude'].apply(max).max()
    min_long = df['Longitude'].apply(min).min()
    
    center_lat = min_lat + (max_lat - min_lat)/2
    center_long = min_long + (max_long - min_long)/2
    
    width_lat = abs(max_lat - min_lat)
    width_long = abs(max_long - min_long)
    
    zoom = np.ceil(1.3 * (1 / max(width_lat, width_long)))
    zoom = max(zoom, 12)
    zoom = min(zoom, 17)
    
    return (center_lat, center_long, zoom)



In [6]:
# Generación del mapa con los elementos de la entrada
# layers: lista de conjuntos de datasets
# mode: 'C' = cultivo, 'P' = Propietario
# initial: '' = posición y zoom por defecto, '' = cálculo automático

def GenerateMap(layers, mode='', initial=''):

    # Consolido los layers en un único conjunto de dataframes
    polygons = pd.DataFrame(columns=['CatReferencia', 'CatRecinto', 'Latitude', 'Longitude', 'CatMunicipio', 'CatPoligono', 'CatParcela', 'VariedadCultivo', 'NombreParcela', 'NombreCultivo', 'ColorCultivo', 'NombrePropietario', 'ColorPropietario', 'Origen'])
    points = pd.DataFrame(columns=['Latitude', 'Longitude', 'Tipo', 'Etiqueta', 'NombreCultivo', 'ColorCultivo', 'Origen'])
    markers = pd.DataFrame(columns=['Latitude', 'Longitude', 'Tipo', 'Etiqueta', 'NombreCultivo', 'ColorCultivo', 'Origen'])
    lines = pd.DataFrame(columns=['Latitude', 'Longitude', 'Tipo', 'Etiqueta', 'NombreCultivo', 'ColorCultivo', 'Origen'])

    for layer in layers:
        if layer.get('parcelas') is not None: polygons = pd.concat([polygons, layer.get('parcelas')], axis=0, ignore_index=True)
        if layer.get('puntos') is not None: points = pd.concat([points, layer.get('puntos')], axis=0, ignore_index=True)
        if layer.get('markers') is not None: markers = pd.concat([markers, layer.get('markers')], axis=0, ignore_index=True)
        if layer.get('lines') is not None: lines = pd.concat([lines, layer.get('lines')], axis=0, ignore_index=True)

    polygons.fillna('', inplace=True)
    points.fillna('', inplace=True)
    markers.fillna('', inplace=True)   
    lines.fillna('', inplace=True)     
        
    #polygons.drop_duplicates(inplace=True, ignore_index=True)
    points.drop_duplicates(inplace=True, ignore_index=True)
    markers.drop_duplicates(inplace=True, ignore_index=True)
    lines.drop_duplicates(inplace=True, ignore_index=True)
        
    # Calculo el marco inicial
    if (initial == 'auto') & (polygons is not None):
        initial_lat, initial_long, initial_zoom = GetMapInitialSizeKML(polygons)
        print(initial_lat, initial_long, initial_zoom)
    else:
        initial_lat = INITIAL_LAT
        initial_long = INITIAL_LONG
        initial_zoom = INITIAL_ZOOM
    
    s_mode = mode.lower()
    
    # Genero el mapa
    gmap = CustomGoogleMapPlotter(initial_lat, initial_long, initial_zoom, map_type='satellite')
    
    if points is not None:
        gmap.AddMapPoints(points, s_mode)
    
    if polygons is not None:
        gmap.AddMapPolygons(polygons, s_mode)    
        
    if lines is not None:
        gmap.AddMapLines(lines, s_mode)
        
    if markers is not None:
        gmap.AddMapMarkers(markers, s_mode)
    
    if s_mode == 'c':
        s_mapname = 'mapa_cultivos.html'
    elif s_mode == 'p':
        s_mapname = 'mapa_propietarios.html'
    else:
        s_mapname = 'mapa.html'
        
    gmap.draw(MAPS_PATH + '/' + s_mapname)    

    # Abrir mapa en el navegador
    url = "file://" + MAPS_PATH + '/' + s_mapname
    webbrowser.open_new_tab(url)


    
    
def GenerateMapLayers(parcels=None, lines=None, points=None, markers=None, mode='', initial=''):

    # Calculo el marco inicial
    if (initial == 'auto') & (parcels is not None):
        initial_lat, initial_long, initial_zoom = GetMapInitialSizeKML(polygons)
        print(initial_lat, initial_long, initial_zoom)
    else:
        initial_lat = INITIAL_LAT
        initial_long = INITIAL_LONG
        initial_zoom = INITIAL_ZOOM
    
    s_mode = mode.lower()
    
    # Genero el mapa
    gmap = CustomGoogleMapPlotter(initial_lat, initial_long, initial_zoom, map_type='satellite')
    
    if points is not None:
        gmap.AddMapPoints(points, s_mode)
    
    if parcels is not None:
        gmap.AddMapPolygons(parcels, s_mode)    
        
    if lines is not None:
        gmap.AddMapLines(lines, s_mode)
        
    if markers is not None:
        gmap.AddMapMarkers(markers, s_mode)
    
    if s_mode == 'c':
        s_mapname = 'mapa_cultivos.html'
    elif s_mode == 'p':
        s_mapname = 'mapa_propietarios.html'
    else:
        s_mapname = 'mapa.html'
        
    gmap.draw(MAPS_PATH + '/' + s_mapname)    

    # Abrir mapa en el navegador
    url = "file://" + MAPS_PATH + '/' + s_mapname
    webbrowser.open_new_tab(url)
    

### Ejecución

In [21]:
layers = ['caza']

# Carga de las parcelas
df_parcels = LoadDatasetsParcels(layers)

# Carga de líneas
#df_lines = LoadDatasetsLines(layers)

# Carga de los puntos
df_points = LoadDatasetsPoints(layers)

# Carga de los marcadores
df_markers = LoadDatasetsMarkers(layers)

# Cargo de MyTrack
#df_mytrack_p, df_mytrack_m = LoadDatasetMyTrack()


# Genero el mapa
GenerateMapLayers(parcels=df_parcels, lines=df_points, markers=df_markers, mode='c')


In [8]:
#p = df_parcelas[(df_parcelas['Ref']=='16259A037001760000ZX') & (df_parcelas['Sub'] == 'f')]
#GenerateMapComp(polygons = p)

#df_parcelas[['Ref', 'Sub']][df_parcelas['VariedadCultivo'] == '']

In [10]:
df_parcels.tail(80)

,CatReferencia,CatRecinto,Latitude,Longitude,CatMunicipio,CatPoligono,CatParcela,VariedadCultivo,NombreParcela,NombreCultivo,ColorCultivo,NombrePropietario,ColorPropietario,Origen
156,16259A036003660000ZZ,a,"[39.6793032909897, 39.6792002316937, 39.679170...","[-2.94128812139778, -2.94114676965321, -2.9410...",Villamayor,36.0,366.0,Uva blanca – Airén,Parcela,Viña,limegreen,María Rosa,hotpink,layer-parcelas
157,16259A037001190000ZH,0,"[39.680423664299, 39.679879124244, 39.67971397...","[-2.9188328233716, -2.91837915377758, -2.91826...",,,,,,,,,,
158,16259A037001230000ZW,0,"[39.6808704609313, 39.680802965212, 39.6805593...","[-2.92036630234352, -2.92035378596911, -2.9202...",,,,,,,,,,
159,16259A037001360000ZM,b,"[39.6769958717194, 39.6767439787304, 39.676621...","[-2.92501278750989, -2.92478149154515, -2.9246...",Villamayor,37.0,136.0,Pinos,Casa del ganado/Pedazo de la sorda,Forestal,darkturquoise,José Ignacio,limegreen,layer-parcelas
160,16259A037001360000ZM,a,"[39.6773193167845, 39.6772536207276, 39.677101...","[-2.92455009594256, -2.92453325981809, -2.9244...",Villamayor,37.0,136.0,Uva blanca – Airén,Pedazo de la sorda,Viña,limegreen,José Ignacio,limegreen,layer-parcelas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,45027A504000360000BR,0,"[39.8214215151452, 39.8201694902814, 39.821550...","[-3.10007356527078, -3.09887806309051, -3.0962...",,,,,,,,,,
232,45027A507000890000BM,0,"[39.7961933078385, 39.796187274638, 39.7955451...","[-3.08874868377611, -3.08874400383195, -3.0883...",,,,,,,,,,
233,45027A507001060000BW,0,"[39.8099535796623, 39.8098836277997, 39.809279...","[-3.09730263229841, -3.0972351239688, -3.09665...",,,,,,,,,,
234,45027A507001580000BD,0,"[39.8159148380404, 39.8149445174824, 39.814957...","[-3.09658941637215, -3.09584708699352, -3.0958...",,,,,,,,,,
